In [3]:
import numpy as np
import os
import pandas as pd
from google.cloud import storage


In [4]:
base_dir = "../../raw_data/"


In [27]:
dfs = []
for i in range(1, 7):
    murs_path = os.path.join(base_dir, f"maquette{i}", f"murs{i}.csv")
    if os.path.exists(murs_path):
        # Cherche la ligne d'en-tête réelle
        with open(murs_path, encoding="utf-8") as f:
            for idx, line in enumerate(f):
                if line.startswith("Id;"):
                    header_row = idx
                    break
        try:
            df = pd.read_csv(murs_path, sep=';', decimal=",", header=header_row)
            dfs.append(df)
            print(f"Chargé : {murs_path} ({df.shape[0]} lignes, {df.shape[1]} colonnes)")
            print(df.head())
        except Exception as e:
            print(f"Erreur de parsing : {murs_path} -> {e}")
    else:
        print(f"Fichier non trouvé : {murs_path}")

if dfs:
    murs_concat = pd.concat(dfs, ignore_index=True)
    print(f"Total concaténé : {murs_concat.shape[0]} lignes, {murs_concat.shape[1]} colonnes")
else:
    murs_concat = pd.DataFrame()
    print("Aucun fichier murs.csv trouvé.")

murs_concat.head()

Chargé : ../../raw_data/maquette1/murs1.csv (1589 lignes, 146 colonnes)
       Id 011EC_Lot 012EC_Ouvrage 013EC_Localisation 014EC_Mode Constructif  \
0  779920        GO           MUR          INTERIEUR           POUTRE VOILE   
1  780111        GO           MUR          INTERIEUR           POUTRE VOILE   
2  780661        GO           MUR          INTERIEUR                 BANCHE   
3  783007        GO           MUR          INTERIEUR                 BANCHE   
4  783554        GO           MUR          INTERIEUR           POUTRE VOILE   

           Nom  Hauteur  Epaisseur            AI    AS  ...  \
0  Voile BA 20     2.88        0.2 -1.378800e-12  2.88  ...   
1  Voile BA 20     2.88        0.2 -1.378800e-12  2.88  ...   
2  Voile BA 20     2.88        0.2 -1.378800e-12  2.88  ...   
3  Voile BA 20     2.88        0.2 -1.378800e-12  2.88  ...   
4  Voile BA 20     2.88        0.2 -1.378800e-12  2.88  ...   

   Code d'assemblage  Retournement aux insertions  \
0                NaN 

,Id,011EC_Lot,012EC_Ouvrage,013EC_Localisation,014EC_Mode Constructif,Nom,Hauteur,Epaisseur,AI,AS,...,EIF_STR - Impact,EC_Type de Mur,ID MONTAGE,Désignation système,Réf DT,Nature_Ouvrage,Batiment,Mur armé,Affichage poteau,NIVEAU_STRUCTURE
0,779920,GO,MUR,INTERIEUR,POUTRE VOILE,Voile BA 20,2.88,0.2,-1.378800e-12,2.88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,780111,GO,MUR,INTERIEUR,POUTRE VOILE,Voile BA 20,2.88,0.2,-1.378800e-12,2.88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,780661,GO,MUR,INTERIEUR,BANCHE,Voile BA 20,2.88,0.2,-1.378800e-12,2.88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,783007,GO,MUR,INTERIEUR,BANCHE,Voile BA 20,2.88,0.2,-1.378800e-12,2.88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,783554,GO,MUR,INTERIEUR,POUTRE VOILE,Voile BA 20,2.88,0.2,-1.378800e-12,2.88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:

if dfs:
    murs_concat = pd.concat(dfs, ignore_index=True)
    print(f"Total concaténé : {murs_concat.shape[0]} lignes, {murs_concat.shape[1]} colonnes")
else:
    murs_concat = pd.DataFrame()
    print("Aucun fichier murs.csv trouvé.")


Total concaténé : 2867 lignes, 160 colonnes


In [29]:
#murs_concat['013EC_Localisation'].unique()
murs_concat.columns

Index(['Id', '011EC_Lot', '012EC_Ouvrage', '013EC_Localisation',
       '014EC_Mode Constructif', 'Nom', 'Hauteur', 'Epaisseur', 'AI', 'AS',
       ...
       'EIF_STR - Impact', 'EC_Type de Mur', 'ID MONTAGE',
       'Désignation système', 'Réf DT', 'Nature_Ouvrage', 'Batiment',
       'Mur armé', 'Affichage poteau', 'NIVEAU_STRUCTURE'],
      dtype='object', length=160)

In [37]:
murs_concat['011EC_Lot'].unique()


array(['GO', nan], dtype=object)

In [31]:
colonnes_a_garder = [
    "Epaisseur",
    "Sols en intersection",
    "Sols coupés (u)",
    "Sols coupants (u)",
    "Sol au-dessus",
    "Sol en-dessous",
    "Fenêtres",
    "Portes",
    "Ouvertures",
    "Murs imbriqués",
    "Mur multicouche",
    "Profil modifié",
    "Extension inférieure",
    "Extension supérieure",
    "Partie inférieure attachée",
    "Partie supérieure attachée",
    "Décalage supérieur",
    "Décalage inférieur",
    "Matériau structurel"
]

# On ne garde que les colonnes présentes dans le DataFrame
murs_concat = murs_concat[[col for col in colonnes_a_garder if col in murs_concat.columns]]

In [34]:
murs_concat["Matériau structurel"].unique()

array(['ECSA - Béton Voiles', 'ECSA - Béton Non porteur', 'EC - Béton',
       '<Par catégorie>', 'Maçonnerie - Voile BA',
       'Blocs béton manufacturés', 'C25/30', 'Maçonnerie'], dtype=object)

In [38]:
murs_concat.keys().to_list()

['Id',
 '011EC_Lot',
 '012EC_Ouvrage',
 '013EC_Localisation',
 '014EC_Mode Constructif',
 'Nom',
 'Hauteur',
 'Epaisseur',
 'AI',
 'AS',
 'Sols en intersection',
 'Sols coupés (u)',
 'Sols coupés (Ids)',
 'Sols coupants (u)',
 'Sols coupants (Ids)',
 'Sol au-dessus',
 'Sol en-dessous',
 'Fenêtres',
 'Portes',
 'Ouvertures',
 'Murs imbriqués',
 'Mur multicouche',
 'Mur empilé',
 'Profil modifié',
 'Image',
 'Catégorie',
 'Section',
 "Type prédéfini d'IFC",
 'Exporter au format IFC sous',
 'Exporter au format IFC',
 'IfcGUID',
 'A une association',
 "Enrobage d'armature - Autres faces",
 "Enrobage d'armature - Face intérieure",
 "Enrobage d'armature - Face extérieure",
 'Variantes',
 'Extension inférieure',
 'Extension supérieure',
 'Volume',
 'Surface',
 'Phase de démolition',
 'Phase de création',
 'Commentaires',
 'Longueur',
 'Famille et type',
 'Famille',
 'Type',
 'Nom de la famille',
 'Nom du type',
 'ID du type',
 'Lié au volume',
 'Structure',
 'Identifiant',
 'Ligne de justific

In [49]:
murs_concat.keys().to_list()

['Id',
 '011EC_Lot',
 '012EC_Ouvrage',
 '013EC_Localisation',
 '014EC_Mode Constructif',
 'Nom',
 'Hauteur',
 'Epaisseur',
 'AI',
 'AS',
 'Sols en intersection',
 'Sols coupés (u)',
 'Sols coupés (Ids)',
 'Sols coupants (u)',
 'Sols coupants (Ids)',
 'Sol au-dessus',
 'Sol en-dessous',
 'Fenêtres',
 'Portes',
 'Ouvertures',
 'Murs imbriqués',
 'Mur multicouche',
 'Mur empilé',
 'Profil modifié',
 'Image',
 'Catégorie',
 'Section',
 "Type prédéfini d'IFC",
 'Exporter au format IFC sous',
 'Exporter au format IFC',
 'IfcGUID',
 'A une association',
 "Enrobage d'armature - Autres faces",
 "Enrobage d'armature - Face intérieure",
 "Enrobage d'armature - Face extérieure",
 'Variantes',
 'Extension inférieure',
 'Extension supérieure',
 'Volume',
 'Surface',
 'Phase de démolition',
 'Phase de création',
 'Commentaires',
 'Longueur',
 'Famille et type',
 'Famille',
 'Type',
 'Nom de la famille',
 'Nom du type',
 'ID du type',
 'Lié au volume',
 'Structure',
 'Identifiant',
 'Ligne de justific

In [75]:
all_features = murs_concat.keys().to_list()
to_keep = ['Hauteur','Epaisseur','Sol au-dessus','Sol en-dessous','Fenêtres','Portes','Ouvertures']
to_exp =  ['Volume', 'Surface','Hauteur', 'Longueur','Largeur']
family =  ['Famille et type', 'Famille']
y = ["Matériau structurel"]

In [78]:
y = murs_concat["Matériau structurel"]
X = murs_concat[to_exp]

In [111]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.3, random_state = 42
)


In [112]:
#minmax scaler test:
from sklearn.preprocessing import MinMaxScaler

minmax_scaler = MinMaxScaler()
X_train_scaled = minmax_scaler.fit_transform(X_train)
X_test_scaled = minmax_scaler.transform(X_test)


In [116]:
import seaborn as sns
import matplotlib as plt

ModuleNotFoundError: No module named 'seaborn'

In [108]:
from xgboost import XGBRegressor


In [114]:

xgb_reg = XGBRegressor(max_depth=10, n_estimators=100, learning_rate=0.1)

xgb_reg.fit(X_train_scaled, y_train,
    #eval_set=[(X_train, y_train), (X_val, y_val)],
    #early_stopping_rounds=5
)

#y_pred = xgb_reg.predict(X_val)

ValueError: could not convert string to float: 'ECSA - Béton Voiles'